In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.patheffects

# =============================================
#  CONFIG & DATA LOADING
# =============================================
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

df = pd.read_csv("stats.csv")

metrics = {
    'Turnaround_Time': 'Turnaround Time (k-ticks)',
    'Waiting_Time': 'Waiting Time (k-ticks)',
    'Response_Time': 'Response Time (k-ticks)',
    'Total_Run_Time': 'Total Run Time (k-ticks)',
    'Context_Switches': 'Context Switches',
    'CPU_Share': 'CPU Share (%)'
}

metric_sd = {
    'Turnaround_Time': 'Turnaround_Time_SD',
    'Waiting_Time': 'Waiting_Time_SD',
    'Response_Time': 'Response_Time_SD',
    'Total_Run_Time': 'Total_Run_Time_SD',
    'Context_Switches': 'Context_Switches_SD',
    'CPU_Share': 'CPU_Share_SD'
}

benchmarks = [
    'short_io', 'long_io', 'short_cpu',
    'long_cpu_child', 'cpu_io_child', 'stress_child'
]

benchmark_labels = {
    'short_io': 'Short I/O',
    'long_io': 'Long I/O',
    'short_cpu': 'Short CPU',
    'long_cpu_child': 'Long CPU (4 procs)',
    'cpu_io_child': 'CPU/IO Mix (10 procs)',
    'stress_child': 'Stress (32 procs)'
}

child_benchmarks = ['long_cpu_child', 'cpu_io_child', 'stress_child']

schedulers = ['RR', 'FIFO', 'EEVDF']
colors = {'RR': '#3498db', 'FIFO': '#e74c3c', 'EEVDF': '#2ecc71'}

log_metrics = {'Waiting_Time', 'Response_Time', 'Context_Switches', 'Total_Run_Time', 'Turnaround_Time'}

metric_list_ordered = [
    ("Turnaround_Time", "Turnaround Time (k-ticks)"),
    ("Waiting_Time", "Waiting Time (k-ticks)"),
    ("Response_Time", "Response Time (k-ticks)"),
    ("Total_Run_Time", "Total Run Time (k-ticks)"),
    ("Context_Switches", "Context Switches"),
    ("CPU_Share", "CPU Share (%)")
]

# =============================================
#  MAIN COMPARISON PLOTS (NO ERROR BARS)
# =============================================
def plot_main_metric(df, metric_key, metric_label):
    plt.figure(figsize=(12,6))
    x = np.arange(len(benchmarks))
    width = 0.25

    for i, sched in enumerate(schedulers):
        vals = []
        for b in benchmarks:
            row = df[(df['Benchmark'] == b) & (df['Scheduler'] == sched)]
            v = row[metric_key].values[0] if not row.empty else 0
            if metric_key in log_metrics:
                v = max(v, 1e-3)
            vals.append(v)

        bars = plt.bar(x + i*width, vals, width,
                       label=sched, color=colors[sched], alpha=0.85)

        # --- VERTICAL LABELS INSIDE THE BAR ---
        for bar in bars:
            h = bar.get_height()
            label_y = h / 2  # center of the bar

            plt.text(
                bar.get_x() + bar.get_width() / 2,
                label_y,
                f"{h:.1f}",
                ha="center",
                va="center",
                rotation=90,          # ← rotate text vertically
                fontsize=11,
                fontweight="bold",
                color="white",
                path_effects=[
                    matplotlib.patheffects.Stroke(linewidth=2, foreground="black"),
                    matplotlib.patheffects.Normal()
                ]
            )
    
    plt.xticks(x + width, [benchmark_labels[b] for b in benchmarks], rotation=20, ha='right')
    plt.ylabel(metric_label)
    plt.title(metric_label)
    plt.grid(axis='y', alpha=0.3)
    if metric_key in log_metrics:
        plt.yscale("log")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Generate all main plots
for mk, ml in metrics.items():
    plot_main_metric(df, mk, ml)

# =============================================
#  CHILD BENCHMARK ERROR-BAR PLOTS (SYML0G SCALE)
# =============================================

def plot_child_benchmark(df, benchmark):
    plt.figure(figsize=(12,6))
    x = np.arange(len(metric_list_ordered))
    width = 0.25

    y_minimum = 1e-6   # prevents negative or zero values on symlog

    for i, sched in enumerate(schedulers):
        vals = []
        lower_errs = []
        upper_errs = []

        for metric_key, _ in metric_list_ordered:
            row = df[(df["Benchmark"] == benchmark) & (df["Scheduler"] == sched)]

            value = float(row[metric_key].values[0])
            sd = row[metric_sd[metric_key]].values[0]
            sd = 0 if pd.isna(sd) else sd

            # Prevent negative bar heights on symlog
            value = max(value, y_minimum)

            # ---- FIX: CLAMP LOWER ERROR BARS ----
            upper_err = sd
            lower_err = min(sd, value - y_minimum)

            vals.append(value)
            lower_errs.append(lower_err)
            upper_errs.append(upper_err)

        xpos = x + (i - 1) * width

                # Bars
        bars = plt.bar(
            xpos, vals, width,
            label=sched, color=colors[sched], alpha=0.85
        )

        # --- ASYMMETRIC ERROR BAR FIX ---
        plt.errorbar(
            xpos, vals,
            yerr=[lower_errs, upper_errs],
            fmt="none",
            ecolor="black",
            elinewidth=1.2,
            capsize=5
        )

        # --- VERTICAL LABELS INSIDE THE BAR (SYML0G) ---
        for j, bar in enumerate(bars):
            h = bar.get_height()
            label_y = h / 2

            plt.text(
                bar.get_x() + bar.get_width()/2,
                label_y,
                f"{h:.1f}",
                ha="center",
                va="center",
                rotation=90,          # ← vertical text
                fontsize=11,
                fontweight="bold",
                color="white",
                path_effects=[
                    matplotlib.patheffects.Stroke(linewidth=2, foreground="black"),
                    matplotlib.patheffects.Normal()
                ]
            )
    
    plt.xticks(x, [label for _, label in metric_list_ordered], rotation=25, ha="right")
    plt.title(f"{benchmark_labels[benchmark]} — Metrics with Std Dev")
    plt.ylabel("Value (symlog scale)")
    plt.grid(axis="y", alpha=0.3)

    plt.yscale("symlog", linthresh=500)

    plt.legend()
    plt.tight_layout()
    plt.show()

# Generate child-benchmark plots
for bench in child_benchmarks:
    plot_child_benchmark(df, bench)
